<a href="https://colab.research.google.com/github/nicolinhainc/mypipeline/blob/master/Pipeline3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%pwd  

In [0]:
%cd '/content'

In [0]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1XdwGEqKXeKH1NRhZXxCZo5T8mncwYHCC' -O ./Bet-n-dcm2nii.zip
!unzip -q ./Bet-n-dcm2nii.zip
!rm -rf images

In [0]:
# upload the LGG zip file yourself as the link to google drive wasnt working

In [0]:
!unzip -q ./LGG.zip
!rm -rf ./_MACOSX
!rm -rf *.DS_Store
!rm -rf ./LGG/.DS_Store

In [0]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1uBQdQIYcA_ACqycw8ofDzD-1G9YwZaRw' -O ./icbm152t2.zip
!unzip -q ./icbm152t2.zip

In [0]:
NIFTI_PATH = './LGG'

In [0]:
import os, fnmatch

In [0]:
os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'

In [0]:
for subj in os.listdir(NIFTI_PATH):
    subj_path = os.path.join(NIFTI_PATH, subj)
    print ("Working on " + subj_path)
    series = fnmatch.filter(os.listdir(subj_path), '*.nii.gz')
    for ser in ['T2*']: #not sure why I had to add the star here but this was the only way it worked
        nii_file = os.path.join (subj_path, ser)
# First, perform N4 bias correction. Not required, but may improve results. Also must track new names
        new_file = os.path.join (subj_path, 'N4-' + ser)
        cmd = "./N4BiasFieldCorrection -i %s.nii.gz -o %s.nii.gz" % (nii_file, new_file)
        os.system(cmd)
        print (cmd)

In [0]:
#now need to delete the original files from each folder but I cant figure out how

In [0]:
# Next, Register the images to the avg 152 t2 image from MNI
for subj in os.listdir(NIFTI_PATH):
    subj_path = os.path.join(NIFTI_PATH, subj)
    print ("Working on " + subj_path)
    series = fnmatch.filter(os.listdir(subj_path), 'N4-*')
    for ser in ['T2']:   
        MNI_file = './icbm_avg_152_t2_tal_lin.nii'
        T2_file =  os.path.join (subj_path, 'N4-T2*.nii.gz')
        new_file = os.path.join (subj_path, 'Reg-N4-T2.nii.gz')
        cmd = "./flirt -in %s -ref %s  -out %s" % (T2_file, MNI_file, new_file)
        os.system(cmd)
        print (cmd)



In [0]:
#now we need to delete the N4-T2s from each folder

In [0]:
#then test the split directory function below. It didnt separate randomly last time so lets see what it does now

In [0]:
!pip install split-folders
!pip install split-folders tqdm


In [0]:
import split_folders
split_folders.ratio('./LGG', output='./output', seed=1337, ratio=(.8, .2))